# TODO: 
- check learning rates
- check early stopping
- check callbacks
- check corect loss functions
- train last layer
- make vgg16

In [ ]:
import os
import io
import tensorflow as tf
import numpy as np
from azure.storage.blob import BlobServiceClient, ContainerClient
from PIL import Image
import matplotlib.pyplot as plt
import ast
import pandas as pd

from tensorflow.keras import layers, models
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import MobileNetV2


from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

#from tensorflow.keras.layers import Dense, Flatten
#from tensorflow.keras.models import Model, Sequential
#from keras.layers import MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, Flatten
#from keras.models import Model

#from keras.layers import MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, Flatten
#from keras.models import Model
#from tensorflow.keras.callbacks import ModelCheckpoint
#from keras.callbacks import EarlyStopping




### Connect to Azure

In [12]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

### Load Image Paths and Labels

In [13]:
#get filepaths
df = pd.read_csv("METER_ML_test.csv")

#create dictionary with labels and encoded labels
unique_types = df['Type'].unique()
unique_type_encoded = df['Type_encoded'].unique()

type_dict = {}

for i in range(len(unique_types)):
    type_dict[unique_type_encoded[i]] = unique_types[i]

sorted_type_dict = dict(sorted(type_dict.items()))

sorted_type_dict

{0: 'CAFOs',
 1: 'Landfills',
 2: 'Mines',
 3: 'Negative',
 4: 'ProcessingPlants',
 5: 'RefineriesAndTerminals',
 6: 'WWTreatment'}

### Functions 
- create data (input for models)
- plot accuracy and loss of models
- metrices
- print dataset
- print evaluation metrices

In [22]:
image_size=224
channels=3
autotune = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically

def data_split(df):
    """Splits and returns the dataset into training, validation, and test"""
    X_temp, X_test, y_temp, y_test = train_test_split(df['Image_Folder'], df['Type_encoded'], test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)
    #convert labels to array
    y_train = np.array(y_train).tolist()
    y_val = np.array(y_val).tolist()
    y_test = np.array(y_test).tolist()
    #print number of observations per datasets
    print("Nr. Training:",len(X_train),"Nr. Validation:",len(X_val),"Nr. Test:",len(X_test))
    
    return X_train, X_val, X_test, y_train, y_val, y_test


def load_image(path):
    """Load an image from Azure Blob Storage."""
    blob_client = container_client.get_blob_client(path)
    blob_data = blob_client.download_blob().readall()  # Directly read all bytes
    
    return io.BytesIO(blob_data)


def load_and_preprocess_image(path):
    """Loads an image, decodes it to grayscale, resizes, and normalizes it."""
    # Load image
    image_file = load_image(path.numpy().decode('utf-8'))
    # Decode the image to grayscale
    image_tensor = tf.io.decode_image(image_file.getvalue(), channels=channels)
    # Resize the image
    image_resized = tf.image.resize(image_tensor, [image_size, image_size])
    # Normalize the image data
    image_normalized = image_resized / 255.0
    return image_normalized


def process_tensor(path, label):
    """Function to load an image from blob storage, decode, resize, and normalize it."""
    image_normalized = tf.py_function(load_and_preprocess_image, [path], tf.float32)
    # Ensure the shape is set correctly for grayscale
    image_normalized.set_shape([image_size, image_size, channels])
    return image_normalized, label


def create_dataset(filenames, labels, is_training=True):
    """Creates a TensorFlow dataset from filenames and labels."""
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(process_tensor, num_parallel_calls=tf.data.AUTOTUNE)
    #shuffle the data when it is the training dataset
    if is_training:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=1024)
    #creates batches    
    dataset = dataset.batch(256)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset


def plot_history(model):
    """Plots the accuracy and loss of the inputted model."""
    # summarize history for accuracy
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


def print_predictions(model, ds):
    """Predictions based on test dataset."""
    #predict
    for images, labels in ds:
        predictions = model.predict(images)  # Only pass image data
     #print(predictions[:1])
        for pred, label in zip(predictions, labels):
            print("Prediction:", pred, "Actual Label:", label.numpy())# Print the first prediction
        break
    
def create_all_datasets(X_train, X_val, X_test, y_train, y_val, y_test ):
    """Creates train, test, and val datasets by calling the create_dataset function each."""
    train_ds = create_dataset(X_train, y_train)
    test_ds = create_dataset(X_test, y_test, False)
    val_ds = create_dataset(X_val, y_val, False)
    
    return train_ds, test_ds, val_ds

def print_dataset(dataset):
    """Print the plain dataset."""
    for images, labels in dataset.take(1):  # Here, take(1) takes the first batch
        print("Images:", images.numpy())  # Convert tensor to numpy array and print
        print("Labels:", labels.numpy())  # Convert tensor to numpy array and print
        
def plot_model(model): 
    """Plot model with predefined arguments."""
    plot_model(model, 
            to_file='vgg.png',
            show_shapes=True,
            show_dtype=True,
            show_layer_names=True,
            show_layer_activations=True,
            show_trainable=False)
    

def evaluate_cnn(model, data, labels):
    """
    Evaluates a CNN model on given data and labels.
    
    Parameters:
        model (tf.keras.Model): The trained CNN model.
        data (np.array): Input data for evaluation (images).
        labels (np.array): True labels for the data.
    
    Returns:
        dict: A dictionary containing accuracy, precision, recall, F1-score, and confusion matrix.
    """
    # Making predictions
    predictions = model.predict(data)
    predicted_classes = tf.argmax(predictions, axis=1)
    true_classes = tf.argmax(labels, axis=1)
    
    # Calculating metrics
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Confusion Matrix": conf_matrix
    }

# Example usage:
# Assuming `model` is your CNN model and you have `test_images` and `test_labels`
# results = evaluate_cnn(model, test_images, test_labels)
# print(results)


In [23]:
X_train, X_val, X_test, y_train, y_val, y_test = data_split(df)
    
train_ds, test_ds, val_ds = create_all_datasets(X_train, X_val, X_test, y_train, y_val, y_test)

Nr. Training: 692 Nr. Validation: 149 Nr. Test: 149


### MobileNetV2: 
https://github.com/ashrefm/multi-label-soft-f1/blob/master/Multi-Label%20Image%20Classification%20in%20TensorFlow%202.0.ipynb

In [ ]:
IMG_SIZE = 224
CHANNELS = 3

def create_mobilenetv2_model():
    # Load MobileNetV2 from Keras Applications
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Adding custom layers on top of MobileNetV2
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)  # Dropout for regularization
    predictions = Dense(7, activation='softmax')(x)  # Final layer with softmax activation for classification

    # Construct the full model
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

model_mnv2 = create_mobilenetv2_model()

# Assuming train_ds, X_test, y_test are properly defined
history = model_mnv2.fit(train_ds,
                      validation_data = val_ds, 
                      epochs=10)

plot_history(history)
print_predictions(model_mnv2, test_ds)

### Inception.v3: 
https://towardsdatascience.com/understanding-the-amazon-rainforest-with-multi-label-classification-vgg-19-inceptionv3-5084544fb655

In [ ]:
#create inception v3
inceptionv3 = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for layer in inceptionv3.layers:
    layer.trainable = False

# Adding custom layers
x = inceptionv3.output
x = GlobalAveragePooling2D()(x)  # Ensure this reduces all spatial dimensions
x = Dense(4096, activation="relu")(x)
x = Dropout(0.1)(x)
output = Dense(7, activation="softmax")(x)  # Adjust the number of output units to match the number of classes

# Creating the final model
inceptionv3_model = Model(inputs=inceptionv3.input, outputs=output)
inceptionv3_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])

# Set up the model checkpoint
model_checkpoint = ModelCheckpoint('inceptionv3_model.keras', monitor="val_accuracy", verbose=1, save_best_only=True)

# Assuming train_ds, X_test, y_test are properly defined
history = inceptionv3_model.fit(train_ds,
                      validation_data = val_ds, 
                      epochs=10, 
                      callbacks=[model_checkpoint])

plot_history(history)
print_predictions(inceptionv3_model, test_ds)

### VGG16 with transfer learning: 
https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False ## Not trainable weights

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(7, activation='softmax')

vgg16_tf_model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

vgg16_tf_model.compile(
optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   patience=5,  
                   restore_best_weights=True)

history = vgg16_tf_model.fit(train_ds, 
                        validation_data = val_ds, 
                        epochs=10, 
                        batch_size=32, 
                        callbacks=[es])

plot_history(history)
print_predictions(vgg16_tf_model, test_ds)

### ResNet50: 
https://datagen.tech/guides/computer-vision/resnet-50/

In [ ]:
resnet_model = Sequential()

rn50_base = tf.keras.applications.ResNet50(
    weights = "imagenet",
    input_shape=(224,224,3)
    )

for each_layer in rn50_base.layers:

        each_layer.trainable=False

resnet_model.add(rn50_base)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation='relu'))

resnet_model.add(Dense(7, activation='softmax'))

resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(train_ds, validation_data = val_ds, epochs=10)

plot_history(resnet_model)
print_predictions(resnet_model, test_ds)

### Self-trained VGG16
Source: https://medium.com/@siddheshb008/vgg-net-architecture-explained-71179310050f

In [ ]:
_input = Input((224,224,1)) 

conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(4096, activation="relu")(dense1)
output = Dense(7, activation="softmax")(dense2) #adapted number of outputs and outputfunction

vgg16_model  = Model(inputs=_input, outputs=output)

LR = 1e-5 #why?

#compile the model
vgg16_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=LR),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

##provide a model summary
vgg16_model.summary()

#fit the model
vgg16_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=10
)

plot_history(vgg16_model)
print_predictions(vgg16_model, test_ds)